CANINE on TyDi QA jupyter notebook, automated and adapted to colab from the available github code at https://github.com/google-research/language/tree/master/language/canine/tydiqa and https://github.com/google-research-datasets/tydiqa#evaluation

In [ ]:
# For google colab

#from google.colab import drive
#drive.mount('/content/drive')
#mkdir drive/MyDrive/canine
#cd drive/MyDrive/canine

In [ ]:
# For running locally

%mkdir canine
%cd canine

In [ ]:
!git clone https://github.com/google-research/language.git

In [ ]:
!pip3 install --upgrade tensorflow-gpu
!pip3 install absl-py
!pip3 install tf-slim

In [ ]:
!pamac install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2

In [ ]:
%cd ./language/
!wget https://storage.googleapis.com/tydiqa/v1.0/tydiqa-v1.0-dev.jsonl.gz
!wget https://storage.googleapis.com/tydiqa/v1.0/tydiqa-v1.0-train.jsonl.gz
!wget https://storage.googleapis.com/caninemodels/canine-c.zip
!wget https://storage.googleapis.com/caninemodels/canine-s.zip

In [ ]:
!unzip canine-c.zip
!unzip canine-s.zip

In [ ]:
!python3 -m language.canine.tydiqa.prepare_tydi_data \
  --input_jsonl=tydiqa-v1.0-dev.jsonl.gz \
  --output_tfrecord=dev.tfrecord \
  --max_seq_length=2048 \
  --doc_stride=512 \
  --max_question_length=256 \
  --is_training=false

In [ ]:
!python3 -m language.canine.tydiqa.prepare_tydi_data \
  --input_jsonl=tydiqa-v1.0-train.jsonl.gz \
  --output_tfrecord=train_samples.tfrecord \
  --record_count_file=train_samples_record_count.txt \
  --max_seq_length=2048 \
  --doc_stride=512 \
  --max_question_length=256 \
  --include_unknowns=0.1 \
  --is_training=true

  # Google Colab always stopped before the end in this cell so the file could not be generated

In [ ]:
!python3 -m language.canine.tydiqa.run_tydi \
  --model_config_file=canine-c/canine_config.json \
  --init_checkpoint=canine_dir/canine_model.ckpt \
  --train_records_file=train_samples/*.tfrecord \
  --record_count_file=train_samples_record_count.txt \
  --do_train \
  --max_seq_length=2048 \
  --train_batch_size=512 \
  --learning_rate=5e-5 \
  --num_train_epochs=10 \
  --warmup_proportion=0.1 \
  --output_dir=tydiqa_baseline_model-c

  # Google Colab always stopped before the end in this cell so the file could not be generated and prevented me from running the following functions

In [ ]:
!python3 -m language.canine.tydiqa.run_tydi \
  --model_config_file=canine-c/canine_config.json \
  --init_checkpoint=tydiqa_baseline_model-c \
  --predict_file=tydiqa-v1.0-dev.jsonl.gz \
  --do_predict \
  --max_seq_length=2048 \
  --max_answer_length=100 \
  --candidate_beam=30 \
  --predict_batch_size=128 \
  --output_dir=tydiqa_baseline_model-c/predict \
  --output_prediction_file=tydiqa_baseline_model-c/predict/pred.jsonl \

In [ ]:
!git clone https://github.com/google-research-datasets/tydiqa.git
!python3 language.canine.tydiqa.tydi_eval.py \
  --gold_path=tydiqa-v1.0-dev.jsonl.gz \
  --predictions_path=tydiqa_baseline_model-c/predict/pred.jsonl

In [ ]:
!python3 -m language.canine.tydiqa.run_tydi \
  --model_config_file=canine-s/canine_config.json \
  --init_checkpoint=canine_dir/canine_model.ckpt \
  --train_records_file=train_samples/*.tfrecord \
  --record_count_file=train_samples_record_count.txt \
  --do_train \
  --max_seq_length=2048 \
  --train_batch_size=512 \
  --learning_rate=5e-5 \
  --num_train_epochs=10 \
  --warmup_proportion=0.1 \
  --output_dir=tydiqa_baseline_model-s

In [ ]:
!python3 -m language.canine.tydiqa.run_tydi \
  --model_config_file=canine-s/canine_config.json \
  --init_checkpoint=tydiqa_baseline_model-s \
  --predict_file=tydiqa-v1.0-dev.jsonl.gz \
  --do_predict \
  --max_seq_length=2048 \
  --max_answer_length=100 \
  --candidate_beam=30 \
  --predict_batch_size=128 \
  --output_dir=tydiqa_baseline_model-s/predict \
  --output_prediction_file=tydiqa_baseline_model-s/predict/pred.jsonl \

In [ ]:
!python3 language.canine.tydiqa.tydi_eval.py \
  --gold_path=tydiqa-v1.0-dev.jsonl.gz \
  --predictions_path=tydiqa_baseline_model-s/predict/pred.jsonl

In [ ]:
# Maybe an interpretation of the outputs here but I didn't get this far.